## What is Zero-Shot Learning?

- <span style="color:green"> **Zero-Shot Learning** </span> is a concept, that a model when trained on enough unlabeled data (unsupervised learning) is able to generalize/ recognize at inference time even though the model was not trained on the inference data. This can be used in NLP, Images etc.
- <span style="color:green"> **Zero-Shot Learning** </span> is a setup in which a model can learn to recognize things that it hasn’t explicitly seen before in training.

#### Load useful libraries and data

In [1]:
from transformers import pipeline
import pandas as pd
import numpy as np
from tqdm import tqdm


c:\Users\tranq\Desktop\neue_fische\ds-intro-to-NLP\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
data = pd.read_csv(
    "data/SMSSpamCollection.txt",
    encoding="utf-8",
    header=None,
    delimiter="\t",
    names=["target", "text"],
)

In [3]:
data.head(5)

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
print(f'There are {data.shape[0]} rows in the dataset')

There are 5572 rows in the dataset


### Preparing the pipeline in one-line of code!

In [5]:
classifier = pipeline("zero-shot-classification",device = 0)

No model was supplied, defaulted to FacebookAI/roberta-large-mnli and revision 130fb28 (https://huggingface.co/FacebookAI/roberta-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\tranq\Desktop\neue_fische\ds-intro-to-NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

### Making Predictions

This model works best with informative labels, spam/ham are not so informative. Using spam/ham leads to a Hamming loss of 53% vs using click bait/written by humans leading to 19%

Can you find better label descriptions?

In [6]:
category_map = {"spam":"click bait", "ham":"written by humans"}

In [7]:
candidate_labels = list(category_map.values())
predictedCategories = []
trueCategories = []
for i in tqdm(range(100)):
    text = data.iloc[i,]['text']
    cat = [data.iloc[i,]['target']]
    res = classifier(text, candidate_labels, multi_label=False)
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = next(k for i, (k,v) in enumerate(sorted_dict.items()))

    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)

100%|██████████| 100/100 [01:08<00:00,  1.47it/s]


In [8]:
for y_true, y_pred in zip(trueCategories[:3], predictedCategories[:3]):
    print(f'True Categories {y_true}')
    print(f'Predicted Categories {y_pred}')
    print('#'*50)

True Categories ['written by humans']
Predicted Categories written by humans
##################################################
True Categories ['written by humans']
Predicted Categories written by humans
##################################################
True Categories ['click bait']
Predicted Categories written by humans
##################################################


### Hamming Loss
The Hamming loss is the fraction of labels that are incorrectly predicted.

In [9]:
from sklearn.metrics import hamming_loss
print(f'The hamming loss is {hamming_loss(trueCategories,predictedCategories):.4f} compared to 0.0237 from the last trained model in Notebook 1')

The hamming loss is 0.1900 compared to 0.0237 from the last trained model in Notebook 1


## Check your understanding

Can you find better "labels" for the category description improve the Hamming Loss?

In [16]:
category_map = {"spam": "spam", "ham": "human"}

candidate_labels = list(category_map.values())
predictedCategories = []
trueCategories = []
for i in tqdm(range(100)):
    text = data.iloc[i,]['text']
    cat = [data.iloc[i,]['target']]
    res = classifier(text, candidate_labels, multi_label=False)
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = next(k for i, (k,v) in enumerate(sorted_dict.items()))

    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)

100%|██████████| 100/100 [01:06<00:00,  1.50it/s]


In [17]:
for y_true, y_pred in zip(trueCategories[:3], predictedCategories[:3]):
    print(f'True Categories {y_true}')
    print(f'Predicted Categories {y_pred}')
    print('#'*50)

print(f'The hamming loss is {hamming_loss(trueCategories,predictedCategories):.4f} compared to 0.0237 from the last trained model in Notebook 1')

True Categories ['human']
Predicted Categories human
##################################################
True Categories ['human']
Predicted Categories human
##################################################
True Categories ['spam']
Predicted Categories human
##################################################
The hamming loss is 0.1700 compared to 0.0237 from the last trained model in Notebook 1
